In [1]:
import pandas as pd

In [2]:
file = "D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\train_E6oV3lV.csv"
test = 'D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\test_tweets_anuFYb8.csv'
tweets = pd.read_csv(file)
test_tweets=pd.read_csv(test)

In [3]:
print(tweets.shape, test_tweets.shape)

(31962, 3) (17197, 2)


In [4]:
tweets[tweets.label==1].count()

id       2242
label    2242
tweet    2242
dtype: int64

In [5]:
tweets1=tweets[tweets.label==1]
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)
#tweets=tweets.append(tweets1)

In [6]:
label=tweets.label

In [7]:
tweets[tweets.label==1].count()

id       6726
label    6726
tweet    6726
dtype: int64

In [8]:
tweets_train_test=tweets.append(test_tweets)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [9]:
cleaned_tweet = tweets_train_test.tweet.str.replace("[^a-zA-Z]", " ")

In [10]:
cleaned_tweet=cleaned_tweet.apply(lambda x :" ".join(w for w in x.split() if w != 'user' ))

In [11]:
cleaned_tweet.count()

53643

In [12]:
from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

In [13]:
stem=PorterStemmer()

In [14]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( stem.stem(w) for w in x.split()))

In [15]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( w for w in x.split() if len(w)>2))

In [16]:
cleaned_tweet.count()

53643

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweet)
sequences = tokenizer.texts_to_sequences(cleaned_tweet)

C:\Amrendra\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
length = []
for x in cleaned_tweet:
    length.append(len(x.split()))
max(length)

25

In [19]:
from keras.preprocessing.sequence import pad_sequences
word_index=pad_sequences(sequences, maxlen=25, padding = 'pre', truncating='pre')

In [20]:
word_index.shape

(53643, 25)

In [21]:
train = word_index[:36446,:]
test = word_index[36446:,:]

In [22]:
test[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,  8157,
       31860,  2101,   988,  2568, 31861,   136, 31862])

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
label.shape

(36446,)

In [25]:
x_train,x_valid,y_train, y_valid = train_test_split(train,label,test_size=0.2,random_state=3, stratify=label)

In [26]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(29156, 25)
(29156,)
(7290, 25)
(7290,)


In [27]:
#n_class=2
#y_train = keras.utils.to_categorical(y_train,n_class)
#y_test = keras.utils.to_categorical(y_valid,n_class)

In [28]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as ply
%matplotlib inline

In [29]:
output_dir = 'model_tweet_output/cnn'
epochs = 5
batch_size = 128
n_dim = 128
n_unique_words = 50000
n_words_to_skip = 5
max_review_length = 25
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_dense = 256
dropout = 0.2

n_conv = 256
k_conv = 3

In [30]:
model = Sequential()
model.add(Embedding(n_unique_words,n_dim, input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(n_dense,activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 128)           6400000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 256)           98560     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total para

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")

In [34]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [35]:
model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid,y_valid), callbacks=[modelcheckpoint])

Train on 29156 samples, validate on 7290 samples
Epoch 1/5
29156/29156 [==============================] - 517s 18ms/step - loss: 0.2579 - acc: 0.8962 - val_loss: 0.1223 - val_acc: 0.9543
Epoch 2/5
29156/29156 [==============================] - 512s 18ms/step - loss: 0.0612 - acc: 0.9797 - val_loss: 0.0656 - val_acc: 0.9767
Epoch 3/5
29156/29156 [==============================] - 510s 17ms/step - loss: 0.0175 - acc: 0.9943 - val_loss: 0.0733 - val_acc: 0.9783
Epoch 4/5
29156/29156 [==============================] - 514s 18ms/step - loss: 0.0062 - acc: 0.9980 - val_loss: 0.0795 - val_acc: 0.9808
Epoch 5/5
29156/29156 [==============================] - 513s 18ms/step - loss: 0.0033 - acc: 0.9991 - val_loss: 0.0655 - val_acc: 0.9866


In [48]:
model.load_weights(output_dir+'/weights.05.hdf5')

In [49]:
y_hat = model.predict(test)

In [50]:
y_hat = pd.DataFrame(y_hat, columns=['Prob']).Prob.apply(lambda x : 1 if x >=0.5 else 0)

In [52]:
y_hat.to_csv('D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\submit_CNN.csv')